In [1]:
import csv
from datetime import datetime, timedelta, time
import win32com.client
import requests

# Google Sheets ID and GID for the specific sheet/tab
sheet_id = '1evlTQYFrQgVEXmRB8dvV9UASXXf5mvTdq_-W_6KhQaY'
gid = '0'  # Typically 0 for the first tab; change it if you need a different tab

# Construct the download URL
csv_url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=csv&gid={gid}'


# CSV file path
csv_file = 'deadlines.csv'

# Initialize Outlook
outlook = win32com.client.Dispatch("Outlook.Application")
calendar = outlook.Session.GetDefaultFolder(9)  # 9 is the folder index for Calendar


# Download the CSV file and save it
def download_csv(url, output_file='deadlines.csv'):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Check for errors
        with open(output_file, 'wb') as file:
            file.write(response.content)
        print(f"CSV file downloaded successfully as '{output_file}'")
    except requests.exceptions.RequestException as e:
        print("Error downloading the file:", e)


# Read deadlines from CSV
def get_deadlines_from_csv(file_path):
    deadlines = {}
    rolling_schools = []
    pending_letters_schools = []

    with open(file_path, mode='r') as file:
        reader = csv.DictReader(file)
        today = datetime.today().date()
        next_monday = today + timedelta(days=(7 - today.weekday()))  # Calculate the upcoming Monday

        for row in reader:
            date_str = row['Deadline'].strip()
            school_name = row['Univ']
            rolling = row.get('Rolling', 'No').strip().lower() == 'yes'
            submitted = row.get('Submitted', 'No').strip().lower() == 'yes'
            letter_status = row.get('Letter', '').strip().lower()

            # Pending letters handling
            if submitted and letter_status == 'not yet':
                pending_letters_schools.append(school_name)
            
            if submitted and letter_status == 'complete':
                continue

            if not date_str:
                continue

            if rolling:
                if date_str == '?':
                    rolling_schools.append(school_name)
                else:
                    try:
                        deadline_date = datetime.strptime(date_str, '%m/%d/%Y').date()
                        if deadline_date <= next_monday:
                            rolling_schools.append(school_name)
                    except ValueError:
                        continue

            try:
                deadline_date = datetime.strptime(date_str, '%m/%d/%Y').date()
                if deadline_date not in deadlines:
                    deadlines[deadline_date] = []
                deadlines[deadline_date].append(school_name)
            except ValueError:
                continue
    
    return deadlines, rolling_schools, pending_letters_schools

# Set start and end times for events
def create_or_update_events(deadlines):
    today = datetime.today().date()
    for deadline, schools in deadlines.items():
        event_date = deadline - timedelta(days=1)
        if event_date < today:
            continue
        
        subject = f"Application Deadlines: {event_date.strftime('%Y-%m-%d')}"
        body = f"Schools with deadlines on {deadline}:\n" + "\n".join(schools)

        # Explicitly set start and end times for new events only
        start_time = datetime.combine(event_date, time(0, 30))  # Start at 12:30 AM
        end_time = datetime.combine(event_date, time(1, 0))     # End at 1:00 AM

        event_exists = False
        for item in calendar.Items:
            calendar.Items.Sort("[Start]")

            if item.Subject == subject and item.Start.date() == event_date:
                item.Body = body  # Update body if event exists
                item.ReminderMinutesBeforeStart = 3 * 24 * 60  # Reminder 3 days before
                item.Save()
                event_exists = True
                break

        if not event_exists:
            event = outlook.CreateItem(1)
            event.Subject = subject
            event.Body = body
            event.Start = start_time
            event.End = end_time
            event.ReminderSet = True
            event.ReminderMinutesBeforeStart = 3 * 24 * 60  # Reminder 3 days before
            event.Save()

# Weekly summary without recurrence, stopping after March 31, 2025
# Function to create or update weekly summary events until March 31, 2025
def create_or_update_weekly_summary(deadlines, rolling_schools, pending_letters_schools):
    today = datetime.today().date()
    
    # Determine the first Monday from today
    next_monday = today + timedelta(days=(7 - today.weekday()))
    
    # Define the stop date as the last Monday to create events
    stop_date = datetime(2025, 3, 31).date()
    
    # Loop through each Monday until the stop date
    while next_monday <= stop_date:
        next_sunday = next_monday + timedelta(days=6)
        
        # Filter schools with deadlines in the upcoming week (Monday to Sunday)
        upcoming_schools = {date: schools for date, schools in deadlines.items() if next_monday <= date <= next_sunday}
        
        # Construct the event body text
        body = "Schools with deadlines this week:\n"
        for date, schools in upcoming_schools.items():
            body += f"\n{date.strftime('%Y-%m-%d')}:\n" + "\n".join(schools)
        
        if rolling_schools:
            body += "\n\n\n\nRolling Deadlines (ongoing or passed):\n" + "\n".join(rolling_schools)
        if pending_letters_schools:
            body += "\n\n\n\nSchools with pending letters:\n" + "\n".join(pending_letters_schools)
        
        subject = "Weekly Application Deadline Summary"
        event_date = next_monday
        
        # Check if an event for this Monday already exists, and update if it does
        event_exists = False
        for item in calendar.Items:
            if item.Subject == subject and item.Start.date() == event_date:
                item.Body = body
                item.Save()
                event_exists = True
                break
        
        # Create a new event if it doesn't exist
        if not event_exists:
            event = outlook.CreateItem(1)  # 1 = olAppointmentItem
            event.Subject = subject
            event.Body = body
            event.Start = datetime.combine(event_date, time(0, 30))  # Start at 12:30 AM
            event.End = datetime.combine(event_date, time(1, 0))     # End at 1:00 AM
            event.ReminderSet = True
            event.ReminderMinutesBeforeStart = 60  # Reminder 1 hour before the event
            event.Save()
        
        # Move to the next Monday
        next_monday += timedelta(weeks=1)


# Call the function
download_csv(csv_url)

# Main function
def main():
    deadlines, rolling_schools, pending_letters_schools = get_deadlines_from_csv(csv_file)
    create_or_update_events(deadlines)
    create_or_update_weekly_summary(deadlines, rolling_schools, pending_letters_schools)

if __name__ == '__main__':
    main()


CSV file downloaded successfully as 'deadlines.csv'
